In [ ]:
# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [58]:
# webdriver: Firefox(geckodriver) e Google Chrome(chormedrive)


# PASSO A PASSO

# PASSO 1: ENTRAR NA INTERNET (ABRIR NAVEGADOR)
# Selenium é uma biblioteca

from selenium import webdriver

navegador = webdriver.Chrome() # --> Automatizando o navegador
navegador.get('https://www.google.com/') #--> Copiando o site que quero entrar


In [59]:
# PASSO 2: IMPORTAR A BASE DE DADOS
import pandas as pd

tabela = pd.read_excel('commodities.xlsx')


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Tilápia,9.05,NaN,NaN


In [60]:
# PASSO 3: PARA CADA PRODUTO DA NOSSA BASE


for linha in tabela.index: #---> Vai analisar por linhas da tabela
    
    produto = tabela.loc[linha, 'Produto']
    
    # entrar no site melhor câmbio
    link = f'https://www.melhorcambio.com/{produto}-hoje'#--> O 'f' é para formatar e o produto em '{}' é para variar o nome.
    link = link.replace('ó', 'o').replace('ú','u').replace('ç','c').replace('á','a').replace('ã','a').replace('â','a').replace('é','e')
    # OU PARA FAZER SUBSTITUIÇÕES GERAIS --> Biblioteca: unicodedata
    # link = unicodedata.normalize('NFKD',link).encode('ascii', 'ignore')
   
    
    print(link)
    navegador.get(link)
    
    #.send.keys('meu nome é Bruno') --> se você quiser escrever nele.
    #.click() --> se você quier clicar nele -------------------------------> Colocar no fim do código.
    #.get_attribute() --> se você quiser pegar alguma informação dele.
     
    
    # PASSO 4: PEGAR PREÇO ATUAL DO PRODUTO
    
    # pegar a cotação
    cotacao = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value') #--> procurando o elemento no site a partir do código por trás dele
    # 1,257,43

    cotacao = cotacao.replace('.','')
    # 1.257,43

    cotacao = cotacao.replace(',','.')
    # 1257.43

    #---> 'replace' --> substituição. O número estava com vírgula e precisa substituir por pontos
    cotacao = float(cotacao)
    print(cotacao)

    
    
    # PASSO 5: ATUALIZAR O PREÇO NA BASE DE DADOS
    
    # na coluna Preço Atual, preencher a tabela
    tabela.loc[linha, 'Preço Atual'] = cotacao
    display(tabela)





https://www.melhorcambio.com/Milho-hoje
57.43
https://www.melhorcambio.com/Soja-hoje
133.82
https://www.melhorcambio.com/Boi-hoje
267.75
https://www.melhorcambio.com/Petroleo-hoje
377.53
https://www.melhorcambio.com/Algodao-hoje
390.52
https://www.melhorcambio.com/Acucar-hoje
148.92
https://www.melhorcambio.com/Cafe-hoje
1042.37
https://www.melhorcambio.com/Ouro-hoje
312.63
https://www.melhorcambio.com/Tilapia-hoje
9.64


In [65]:
# PASSO 6: DECIDIR QUAIS PRODUTOS DEVEM SER COMPRADOS

display(tabela)
tabela['Comprar'] = tabela['Preço Atual'] < tabela['Preço Ideal'] #--> Definindo os itens que precisam ser comprados

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,57.43,NaN
1,Soja,163.59,133.82,NaN
2,Boi,282.20,267.75,NaN
3,Petróleo,424.37,377.53,NaN
4,Algodão,497.76,390.52,NaN
5,Açúcar,136.23,148.92,NaN
6,Café,1092.87,1042.37,NaN
7,Ouro,321.77,312.63,NaN
8,Tilápia,9.05,9.64,NaN


In [66]:
# PASSO 7: EXPORTAR A BASE DE DAIS ATUALIZADO

navegador.quit() #--> Vai fechar o navegador ao final

tabela.to_excel('commodities_atualizado.xlsx', index=False) #--> Criando uma nova base de dados atualizada e sem os índices